## __Sparse Linear System__

#### module
  * numpy 
    + object for matrix, verctor and their operations 
  * matplotlib
    + visualization (2D plot)
    



#### sparse matrix format    

* __csr_matrix: Compressed Sparse Row format__
* csc_matrix: Compressed Sparse Column format
* bsr_matrix: Block Sparse Row format
* lil_matrix: List of Lists format
* dok_matrix: Dictionary of Keys format
* coo_matrix: COOrdinate format (aka IJV, triplet format)
* dia_matrix: DIAgonal format
* __(c.f.) ell_matrix: Ellapack format__ 

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import cg as sparse_cg

import matplotlib.pyplot as plt


* csr_matrix(D)
    + with a dense matrix or rank-2 ndarray D  

* csr_matrix(S)
    + with another sparse matrix S (equivalent to S.tocsr())  

* csr_matrix((M, N), [dtype])
    + to construct an empty matrix with shape (M, N) dtype is optional, defaulting to dtype=’d’.  

* csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
    + where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].  

* __csr_matrix((data, indices, indptr), [shape=(M, N)])__
    + is the standard CSR representation where the column indices for row i are stored in indices[indptr[i]:indptr[i+1]] and their corresponding values are stored in data[indptr[i]:indptr[i+1]]. If the shape parameter is not supplied, the matrix dimensions are inferred from the index arrays.




#### Example

In [ ]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

csr_matrix((data, indices, indptr), shape=(3, 3)).toarray()


In [ ]:
N = 8
h = 1/N

In [ ]:
# implement own csr matrix for 1-D Laplase equation with zero dirichlet boundary
# omit end point as boundary points
# number of points of domain inside are N-1
# 2 , 3*(N-3),  2

# index pointer
indptr = [0] + list(range(2, 2+3*(N-2), 3)) + [3*(N-2)+1]

# indices
indices = [[0, 1]] + [[k-1, k, k+1] for k in range(1, N-2)] + [[N-3, N-2]]
indices = [v for ind in indices for v in ind]

# data
data = np.asarray([2, -1] + [-1, 2, -1]*(N-3) + [-1, 2])
# data = [v for d in data for v in d]


In [ ]:
A = csr_matrix((data, indices, indptr), shape=(N-1, N-1))# .toarray()
A

In [ ]:
b = np.ones(N-1)*(h**2)

In [ ]:
u = np.zeros(N+1)

In [ ]:
# index stride ...

# conjugate gradient
u[1:-1], ok = sparse_cg(A, b, tol=1.0e-8)

# numpy.. not available ..
# u[1:-1] = np.linalg.solve(A, b)

In [ ]:
ok

In [ ]:
x = np.linspace(0, 1, N+1, endpoint=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(x, u)
ax.plot(x, 0.5*x*(1-x))
# ax.set_xlim(0.4, 0.6)
# ax.set_ylim(0.11, 0.13)
fig.tight_layout()

In [ ]:
print('{:.5e}'.format(np.linalg.norm(u - 0.5*x*(1-x))))

In [ ]:
def sparse_solve(b):
    N = b.shape[0]-1
    # index pointer
    indptr = [0] + list(range(2, 2+3*(N-2), 3)) + [3*(N-2)+1]

    # indices
    indices = [[0, 1]] + [[k-1, k, k+1] for k in range(1, N-2)] + [[N-3, N-2]]
    indices = [v for ind in indices for v in ind]

    # data
    data = np.asarray([2, -1] + [-1, 2, -1]*(N-3) + [-1, 2])
    # data = [v for d in data for v in d]

    A = csr_matrix((data, indices, indptr), shape=(N-1, N-1)).toarray()
    u = np.zeros(N+1)
    
    # conjugate gradient
    u[1:-1], ok = sparse_cg(A, b[1:-1], tol=1.0e-8)

    return u

In [ ]:
for i in range(14, 18):
    N = 2**i
    h = 1/N
    
    x_ = np.linspace(0, 1, N+1, endpoint=True)
    
    # u = sin(pi*x)
    # repeat function implementation
    b = np.pi*np.pi*np.sin(np.pi*x_)*h*h
    u = sparse_solve(b)
     
    print('error: {:.8e}'.format(np.amax(np.abs(u-np.sin(np.pi*x_)))))